### CS 156a, HW #8, P2-10
Author: Liting Xiao

In [1]:
import numpy as np
from sklearn.svm import SVC

In [2]:
# read in train/test data sets
train_x = np.loadtxt('features.train', usecols=(1,2), unpack=True).T
train_y = np.loadtxt('features.train', usecols=0)
test_x = np.loadtxt('features.test', usecols=(1,2), unpack=True).T
test_y = np.loadtxt('features.test', usecols=0)

#### P2-4

In [3]:
def one_vs_all(train_x, train_y, digit,
               kernel='poly', C=0.01, Q=2):
    # create one_vs_all labels
    digit_y = np.ones_like(train_y)
    digit_y[train_y!=digit] = -1
    
    # SVM to get E_in
    svc = SVC(C=C, kernel=kernel, degree=Q, gamma=1, coef0=1)
    svc.fit(train_x, digit_y)
    digit_pred = svc.predict(train_x)
    N_svm = svc.support_vectors_.shape[0]
    return N_svm, sum((digit_pred!=digit_y)) / float(len(digit_y))

for d in range(0, 10):
    N_svm, E_in = one_vs_all(train_x, train_y, d)
    print('"{} vs all":'.format(d))
    print('    E_in: {:.3f}; The number of support vectors: {}\n'
          .format(E_in, N_svm))

"0 vs all":
    E_in: 0.106; The number of support vectors: 2179

"1 vs all":
    E_in: 0.014; The number of support vectors: 386

"2 vs all":
    E_in: 0.100; The number of support vectors: 1970

"3 vs all":
    E_in: 0.090; The number of support vectors: 1950

"4 vs all":
    E_in: 0.089; The number of support vectors: 1856

"5 vs all":
    E_in: 0.076; The number of support vectors: 1585

"6 vs all":
    E_in: 0.091; The number of support vectors: 1893

"7 vs all":
    E_in: 0.088; The number of support vectors: 1704

"8 vs all":
    E_in: 0.074; The number of support vectors: 1776

"9 vs all":
    E_in: 0.088; The number of support vectors: 1978



2. The highest E_in is: [a] 0 vs all.
3. The lowest E_in is: [a] 1 vs all.
4. The number of SVs difference between "0 vs all" and "1 vs all" is thus: 2179-386=1793, closest to [c].

## P5-6

In [4]:
def data_for_1v1(x, y, digit1, digit2):
    new_y = np.ones_like(y)
    new_y[y==digit2] = -1
    new_y = new_y[(y==digit1) | (y==digit2)]
    new_x = x[(y==digit1) | (y==digit2)]
    return new_x, new_y

def one_vs_one(train_x, train_y, test_x, test_y, 
               digit1, digit2, kernel='poly', C=0.01, Q=2):
    # get 'one_vs_one' ready data sets
    x1, y1 = data_for_1v1(train_x, train_y, digit1, digit2)
    x2, y2 = data_for_1v1(test_x, test_y, digit1, digit2)
    
    # SVM to get E_in
    svc = SVC(C=C, kernel=kernel, degree=Q, gamma=1, coef0=1)
    svc.fit(x1, y1)
    N_svm = svc.support_vectors_.shape[0]
    E_in = sum((svc.predict(x1)!=y1)) / float(len(y1))
    E_out = sum((svc.predict(x2)!=y2)) / float(len(y2))
    return N_svm, E_in, E_out

for C in [0.0001, 0.001, 0.01, 0.1, 1]:
    N_svm, E_in, E_out = one_vs_one(
        train_x, train_y, test_x, test_y, 1, 5, C=C, Q=2)
    print('C={}, Q={}:'.format(C, 2))
    print('    E_in: {:.3f}; E_out: {:.3f}'.format(E_in, E_out))
    print('    The number of support vectors is: {}\n'.format(N_svm))
    
    N_svm, E_in, E_out = one_vs_one(
        train_x, train_y, test_x, test_y, 1, 5, C=C, Q=5)
    print('C={}, Q={}:'.format(C, 5))
    print('    E_in: {:.3f}; E_out: {:.3f}'.format(E_in, E_out))
    print('    The number of support vectors is: {}\n'.format(N_svm))

C=0.0001, Q=2:
    E_in: 0.009; E_out: 0.017
    The number of support vectors is: 236

C=0.0001, Q=5:
    E_in: 0.004; E_out: 0.019
    The number of support vectors is: 26

C=0.001, Q=2:
    E_in: 0.004; E_out: 0.017
    The number of support vectors is: 76

C=0.001, Q=5:
    E_in: 0.004; E_out: 0.021
    The number of support vectors is: 25

C=0.01, Q=2:
    E_in: 0.004; E_out: 0.019
    The number of support vectors is: 34

C=0.01, Q=5:
    E_in: 0.004; E_out: 0.021
    The number of support vectors is: 23

C=0.1, Q=2:
    E_in: 0.004; E_out: 0.019
    The number of support vectors is: 24

C=0.1, Q=5:
    E_in: 0.003; E_out: 0.019
    The number of support vectors is: 25

C=1, Q=2:
    E_in: 0.003; E_out: 0.019
    The number of support vectors is: 24

C=1, Q=5:
    E_in: 0.003; E_out: 0.021
    The number of support vectors is: 21



5. [a] & [b]: The number of support vectors doesn't strictly go down.<br>
   [c]: E_out doesn't always go down as C goes up.<br>
   <b>[d]: This is true.</b>
   <br><br>
6. [a]: E_in is higher at Q=2. <br>
   <b>[b]: This is True. </b> <br>
   [c]: E_in's are the same here. <br>
   [d]: E_out is larger at Q=5.

#### P7-8

In [5]:
def get_Ecv(train_x, train_y, digit1, digit2, N_part,
          kernel='poly', C=0.01, Q=2):
    # partition into N_part folds to get val and train sets
    part_x = np.array_split(train_x, N_part)
    part_y = np.array_split(train_y, N_part)
    E_cv = 0
    for i in range(N_part):
        val_x = part_x[i]
        val_y = part_y[i]
        train_x = np.concatenate(np.delete(part_x, i))
        train_y = np.concatenate(np.delete(part_y, i))
        
        _, _, Err = one_vs_one(train_x, train_y, val_x, val_y,
                               digit1, digit2, C=C)
        E_cv += Err
    return E_cv / float(N_part)

complete_dict = {}
for C in [0.0001, 0.001, 0.01, 0.1, 1]:
    complete_dict[C] = []

C_list = []
for epoch in range(100):
    E_cv = {}    
    for C in [0.0001, 0.001, 0.01, 0.1, 1]:
        E_cv[C] = get_Ecv(train_x, train_y, 1, 5, 10, C=C)
        complete_dict[C].append(E_cv[C])
    C_list.append(min(E_cv, key=E_cv.get))

In [6]:
values, cts = np.unique(C_list, return_counts=True)
C_most = values[np.argmax(cts)]
print('7. The most often selected is: {}'.format(C_most))
Ecv_Cmost = np.mean(complete_dict[C_most])
print('8. The average E_cv for {} is: {}'.format(C_most, Ecv_Cmost))

7. The most often selected is: 0.001
8. The average E_cv for 0.001 is: 0.004436338940129589


#### P9-10

In [7]:
for C in [1e-2, 1, 1e2, 1e4, 1e6]:
    _, E_in, E_out = one_vs_one(train_x, train_y,
        test_x, test_y, 1, 5, kernel='rbf', C=C)
    print('C = {}:'.format(C))
    print('    E_in: {:.3f}; E_out: {:.3f}\n'.format(E_in, E_out))

C = 0.01:
    E_in: 0.004; E_out: 0.024

C = 1:
    E_in: 0.004; E_out: 0.021

C = 100.0:
    E_in: 0.003; E_out: 0.019

C = 10000.0:
    E_in: 0.003; E_out: 0.024

C = 1000000.0:
    E_in: 0.001; E_out: 0.024



9. The lowest E_in happens when C=1e6.
10. The lowest E_out happens when C=100.